In [97]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv('data/training_data.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1], df.iloc[:, 2], test_size=0.2, random_state=42)

# Create an XGBoost regression model
model = xgb.XGBRegressor()

# Reshape the data to fit the model's requirements
X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Generate a range of inputs
inputs = np.linspace(0, 1, num=1000).reshape(-1, 1)

# Predict the outputs for the inputs using the trained model
outputs = model.predict(inputs)

# Find the input with the highest predicted output
highest_output_index = np.argmax(outputs)
highest_output_input = inputs[highest_output_index][0]

print("Input with the highest output:", highest_output_input)


Mean Squared Error: 0.0032769400963140283
Input with the highest output: 0.9879879879879879


In [98]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

df = pd.read_csv('data/training_data.csv')

# Load the input-output pairs from the dataframe
inputs = df.iloc[:, 1].values
outputs = df.iloc[:, 2].values

# Split the data into training and validation sets
train_inputs = inputs[:800]
train_outputs = outputs[:800]
val_inputs = inputs[800:]
val_outputs = outputs[800:]

# Define the neural network architecture
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_inputs, train_outputs, epochs=30, batch_size=200, validation_data=(val_inputs, val_outputs))

# Use the model to predict the output for a given input
input_to_predict = 0.5
predicted_output = model.predict(np.array([input_to_predict]))

Epoch 1/30
4/4 [==============================] - 1s 44ms/step - loss: 0.2181 - val_loss: 0.1786
Epoch 2/30
4/4 [==============================] - 0s 4ms/step - loss: 0.1510 - val_loss: 0.1228
Epoch 3/30
4/4 [==============================] - 0s 4ms/step - loss: 0.1012 - val_loss: 0.0815
Epoch 4/30
4/4 [==============================] - 0s 5ms/step - loss: 0.0651 - val_loss: 0.0520
Epoch 5/30
4/4 [==============================] - 0s 6ms/step - loss: 0.0399 - val_loss: 0.0316
Epoch 6/30
4/4 [==============================] - 0s 7ms/step - loss: 0.0227 - val_loss: 0.0179
Epoch 7/30
4/4 [==============================] - 0s 5ms/step - loss: 0.0120 - val_loss: 0.0101
Epoch 8/30
4/4 [==============================] - 0s 5ms/step - loss: 0.0070 - val_loss: 0.0072
Epoch 9/30
4/4 [==============================] - 0s 5ms/step - loss: 0.0059 - val_loss: 0.0071
Epoch 10/30
4/4 [==============================] - 0s 4ms/step - loss: 0.0068 - val_loss: 0.0078
Epoch 11/30
4/4 [=====================

In [90]:
# Generate a range of input values
inputs_to_test = np.linspace(0, 1, 1000)

# Use the model to predict the output for each input value
predicted_outputs = model.predict(inputs_to_test)

# Find the input that corresponds to the highest predicted output
index_of_max_output = np.argmax(predicted_outputs)
input_with_max_output = inputs_to_test[index_of_max_output]

print(input_with_max_output)

32/32 [==============================] - 0s 463us/step
1.0


In [95]:
def gradient_descent(p2, guess):
    def objective(x):
            input_to_predict = x
            predicted_output = model.predict(np.array([input_to_predict]))
            return predicted_output[0][0]

    # Define the initial input, the learning rate, and the number of iterations
    x = guess
    learning_rate = 0.01
    num_iterations = 100


    iter_vals = []

    beta = 0.99
    
    # Optimize the input using Adam optimization
    for i in range(num_iterations):
        if len(iter_vals) >= 2 and abs(iter_vals[-1] - iter_vals[-2]) < 0.0005:
             break

        # Compute the gradient of the objective function using numerical differentiation
        gradient = (objective(x + 0.001) - objective(x)) / 0.001
       
        x += (learning_rate * gradient) * (beta)**i

        iter_vals.append(x)

        print(x)

    return x, objective(x)

In [96]:
print(gradient_descent('Default', 0.5))

1/1 [==============================] - 0s 10ms/step
0.5054380297660828
1/1 [==============================] - 0s 10ms/step
0.5108219742774963
1/1 [==============================] - 0s 11ms/step
0.5161520793437957
1/1 [==============================] - 0s 12ms/step
0.5214294617027043
1/1 [==============================] - 0s 12ms/step
0.5266534976781844
1/1 [==============================] - 0s 12ms/step
0.5318258601281507
1/1 [==============================] - 0s 10ms/step
0.5369464989536175
1/1 [==============================] - 0s 12ms/step
0.5420164869450692
1/1 [==============================] - 0s 10ms/step
0.5470346750592119
1/1 [==============================] - 0s 10ms/step
0.5520032257909234
1/1 [==============================] - 0s 11ms/step
0.5569220910153179
1/1 [==============================] - 0s 11ms/step
0.5617912339242823
1/1 [==============================] - 0s 10ms/step
0.5666159120165902
1/1 [==============================] - 0s 16ms/step
0.5713923433279751
1/1 [=